# Problem Statement

**(РУ)**  
Подготовить рейтинг городов исходя из среднего показателя оборота на 1 клиента. (формат результата - файл Ms Excel, если решение с помощью питона, то ноутбук)  
Исходные данные:  
- Выгрузка 1 – Ключ контрагента (уникальный идентификатор клиента),Ключ Подразделения контрагента (уникальный идентификатор подразделения), 3-й столбец без названия – оборот (вымышленные данные)  
- Выгрузка 2 – справочник Ключ Подразделения контрагента – город

**(EN)**  
Prepare a city rating based on the average turnover per client. (result format - MS Excel file, if the solution is using Python, then a Jupyter Notebook)  
Initial data:

- Upload 1 - Counterparty Key (unique client identifier), Counterparty Department Key (unique department identifier), 3rd column without a name - turnover (fictitious data)
- Upload 2 - Counterparty Department Key directory - city

# Downloading the Data

In [1]:
import pandas as pd

pd.options.display.max_rows = 100

In [2]:
# libriary for opening old format xls files
!pip install xlrd

In [3]:
extraction_directory = 'Test assignments/PA - city rating'

data_1_csv_path = extraction_directory + '/1_Выгрузка_для_тестового_задания1_1.csv'
data_1_df = pd.read_csv(data_1_csv_path, sep=',', index_col=None)

data_2_xls_path = extraction_directory + '/2_Выгрузка_для_тестового_задания1_2.xls'
data_2_df = pd.read_excel(data_2_xls_path, index_col=None)

In [4]:
data_1_df

,Ключ контрагента,Ключ Подразделения контрагента,Unnamed: 2
0,-1,-1,32 930
1,8,1046,86
2,18,126,3 763
3,32,63,149
4,43,13,297
...,...,...,...
1100383,10024110,1710,33
1100384,10027135,79,1 048
1100385,10034869,54,25
1100386,10050765,10,17


In [5]:
data_1_df = data_1_df.rename(columns={'Unnamed: 2': 'Metric'})
data_1_df

,Ключ контрагента,Ключ Подразделения контрагента,Metric
0,-1,-1,32 930
1,8,1046,86
2,18,126,3 763
3,32,63,149
4,43,13,297
...,...,...,...
1100383,10024110,1710,33
1100384,10027135,79,1 048
1100385,10034869,54,25
1100386,10050765,10,17


In [6]:
data_2_df

,Ключ Подразделения контрагента,Город ОСП контрагента
0,-1,-
1,1,Санкт-Петербург
2,2,Пермь
3,3,Москва
4,4,Архангельск
...,...,...
301,2159,Ялта
302,2164,Нижний Новгород
303,2173,Москва
304,2182,Новосибирск


# Solution

## Prepare Data

In [7]:
# Merge DFs
merged_df = pd.merge(data_1_df, data_2_df,
                     left_on='Ключ Подразделения контрагента',
                     right_on='Ключ Подразделения контрагента',
                     how='left')
merged_df

,Ключ контрагента,Ключ Подразделения контрагента,Metric,Город ОСП контрагента
0,-1,-1,32 930,-
1,8,1046,86,Ачинск
2,18,126,3 763,Чита
3,32,63,149,Киров
4,43,13,297,Уфа
...,...,...,...,...
1100383,10024110,1710,33,Новочебоксарск
1100384,10027135,79,1 048,Забайкальск
1100385,10034869,54,25,Красноярск
1100386,10050765,10,17,Ростов-на-Дону


In [8]:
merged_df['Город ОСП контрагента'] = merged_df['Город ОСП контрагента'].replace('Կนτα', 'Чита')
merged_df['Город ОСП контрагента'] = merged_df['Город ОСП контрагента'].replace('Нет города', 'Нет Данных')
merged_df['Город ОСП контрагента'] = merged_df['Город ОСП контрагента'].replace('-', 'Нет Данных')

In [9]:
merged_df['Город ОСП контрагента'].dtype

dtype('O')

In [10]:
merged_df['Metric'].dtype

dtype('O')

In [11]:
# lets change it to numeric

merged_df['Metric'] = merged_df['Metric'].str.replace(' ', '')
merged_df['Metric'] = pd.to_numeric(merged_df['Metric'], errors='coerce')
merged_df['Metric'].dtype

dtype('float64')

In [12]:
merged_df['Ключ Подразделения контрагента'].dtype

dtype('int64')

In [13]:
merged_df['Ключ контрагента'].dtype

dtype('int64')

## Group Data

In [14]:
# Group merged dataframe
metric = merged_df.groupby('Город ОСП контрагента')
metric

In [15]:
metric = merged_df.groupby('Город ОСП контрагента').agg(turnover = ('Metric', 'sum'), unique_clients = ('Ключ контрагента', 'nunique')).reset_index()
metric

,Город ОСП контрагента,turnover,unique_clients
0,Абакан,761305.0,4542
1,Адлер,545713.0,3147
2,Альметьевск,413887.0,2553
3,Ангарск,390152.0,2368
4,Апатиты,443482.0,2728
...,...,...,...
180,Чита,1246557.0,6753
181,Шахты,316406.0,2042
182,Энгельс,403109.0,2564
183,Ялта,1458.0,7


## Calculate Average Turnover per Client

In [16]:
metric['avg_turnover_per_client'] = metric['turnover'] / metric['unique_clients']
metric

,Город ОСП контрагента,turnover,unique_clients,avg_turnover_per_client
0,Абакан,761305.0,4542,167.614487
1,Адлер,545713.0,3147,173.407372
2,Альметьевск,413887.0,2553,162.117901
3,Ангарск,390152.0,2368,164.760135
4,Апатиты,443482.0,2728,162.566716
...,...,...,...,...
180,Чита,1246557.0,6753,184.593070
181,Шахты,316406.0,2042,154.949070
182,Энгельс,403109.0,2564,157.218799
183,Ялта,1458.0,7,208.285714


There is still 'Город ОСП контрагента' with 'Нет города' values in the data

# Save the data to Excel File

In [17]:
output_directory = extraction_directory + '/city_rating.xlsx'
metric.to_excel(output_directory, index=False)